In [ ]:
pip install tensorflow_addons

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 591.0/591.0 kB 35.0 MB/s eta 0:00:00


In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
mit_test = pd.read_csv('mitbih_test.csv', header=None)
mit_train = pd.read_csv('mitbih_train.csv', header=None)
mit_test.rename(columns={187: "Class"}, inplace=True)
mit_train.rename(columns={187: "Class"}, inplace=True)
print("MIT Train classes: \n", mit_train["Class"].value_counts())
print("\nMIT Test classes: \n", mit_test["Class"].value_counts())
print(mit_train.shape)
print(mit_test.shape)
# Setting Dictionary to define the type of Heartbeat
# N: normal
# S: Supraventricular premature beat
# P: Premature ventricular contraction
# F: Fusion of ventricular and normal beat
# U: Unclassifiable beat
MIT_Outcome = {0.: 'N',
               1.: 'S',
               2.: 'V',
               3.: 'F',
               4.: 'Q'}
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import normalize
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay
from sklearn.utils import resample

import tensorflow as tf
import tensorflow_addons as tfa
from tensorflow import keras
from keras.layers import Dense, Conv1D, MaxPool1D, Flatten, Dropout, InputLayer, LSTM, GRU, BatchNormalization, \
    Bidirectional, Concatenate
from keras.models import Model
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.utils import to_categorical
from sklearn.metrics import accuracy_score

train_mit, valid_mit = train_test_split(mit_train, test_size=0.20, random_state=42)
class_0 = train_mit[train_mit["Class"] == 0]
class_1 = train_mit[train_mit["Class"] == 1]
class_2 = train_mit[train_mit["Class"] == 2]
class_3 = train_mit[train_mit["Class"] == 3]
class_4 = train_mit[train_mit["Class"] == 4]
# upsampling and equalize the number of sample for each class
class_0_upsample = class_0
class_1_upsample = resample(class_1, replace=True, n_samples=10000, random_state=123)
class_2_upsample = resample(class_2, replace=True, n_samples=10000, random_state=124)
class_3_upsample = resample(class_3, replace=True, n_samples=10000, random_state=125)
class_4_upsample = resample(class_4, replace=True, n_samples=10000, random_state=126)

# combine
mit_train = pd.concat([class_0_upsample, class_1_upsample, class_2_upsample, class_3_upsample, class_4_upsample])
train_mit, out_train_mit = mit_train.iloc[:, :187], mit_train.iloc[:, -1]
valid_mit, out_valid_mit = valid_mit.iloc[:, :187], valid_mit.iloc[:, -1]
test_mit, out_test_mit = mit_test.iloc[:, :187], mit_test.iloc[:, -1]
# Normalizing the training, validation & test data
train_mit = normalize(train_mit, axis=0, norm='max')
valid_mit = normalize(valid_mit, axis=0, norm='max')
test_mit = normalize(test_mit, axis=0, norm='max')
# Reshaping the dataframe into a 3-D Numpy array (batch, Time Period, Value)
x_train_mit = train_mit.reshape(len(train_mit), train_mit.shape[1], 1)
x_valid_mit = valid_mit.reshape(len(valid_mit), valid_mit.shape[1], 1)
x_test_mit = test_mit.reshape(len(test_mit), test_mit.shape[1], 1)

# Converting the output into a categorical array
y_train_mit = to_categorical(out_train_mit)
y_valid_mit = to_categorical(out_valid_mit)
y_test_mit = to_categorical(out_test_mit)
len(test_mit)
print("Traing dataset size: ", x_train_mit.shape, " -- Y size: ", y_train_mit.shape)
print("Validation dataset size: ", x_valid_mit.shape, " -- Y size: ", y_valid_mit.shape)
print("Test dataset size: ", x_test_mit.shape, " -- Y size: ", y_test_mit.shape)


def build_conv1d_model_mit(input_shape=(x_train_mit.shape[1], 1)):
    model = keras.models.Sequential()

    input_ = tf.keras.layers.Input(shape=(input_shape))

    conv1_1 = Conv1D(32, 5, padding='same', input_shape=input_shape)(input_)
    conv1_1 = BatchNormalization()(conv1_1)

    conv1_2 = Conv1D(32, 5, padding='same')(conv1_1)
    conv1_2 = BatchNormalization()(conv1_2)
    conv1_2 = tf.keras.layers.ReLU()(conv1_2)

    conv1_3 = Conv1D(32, 5, padding='same')(conv1_2)
    conv1_3 = BatchNormalization()(conv1_3)

    concat_1 = Concatenate()([conv1_1, conv1_3])
    concat_1 = tf.keras.layers.ReLU()(concat_1)
    max_1 = MaxPool1D(5, padding="same")(concat_1)
    #
    conv1_4 = Conv1D(32, 5, padding='same')(max_1)
    conv1_4 = BatchNormalization()(conv1_4)
    conv1_4 = tf.keras.layers.ReLU()(conv1_4)

    conv1_5 = Conv1D(32, 5, padding='same')(conv1_4)
    conv1_5 = BatchNormalization()(conv1_5)

    concat_2 = Concatenate()([max_1, conv1_5])
    concat_2 = tf.keras.layers.ReLU()(concat_2)
    max_2 = MaxPool1D(5, strides=2, padding="same")(concat_2)
    #
    conv1_6 = Conv1D(32, 5, padding='same')(max_2)
    conv1_6 = BatchNormalization()(conv1_6)
    conv1_6 = tf.keras.layers.ReLU()(conv1_6)

    conv1_7 = Conv1D(32, 5, padding='same')(conv1_6)
    conv1_7 = BatchNormalization()(conv1_7)

    concat_3 = Concatenate()([max_2, conv1_7])
    concat_3 = tf.keras.layers.ReLU()(concat_3)
    max_3 = MaxPool1D(5, padding="same")(concat_3)
    #
    conv1_8 = Conv1D(32, 5, padding='same')(max_3)
    conv1_8 = BatchNormalization()(conv1_8)
    conv1_8 = tf.keras.layers.ReLU()(conv1_8)

    conv1_9 = Conv1D(32, 5, padding='same')(conv1_8)
    conv1_9 = BatchNormalization()(conv1_9)

    concat_4 = Concatenate()([max_3, conv1_9])
    concat_4 = tf.keras.layers.ReLU()(concat_4)
    max_4 = MaxPool1D(5, padding="same")(concat_4)
    #
    conv1_10 = Conv1D(32, 5, padding='same')(max_4)
    conv1_10 = BatchNormalization()(conv1_10)
    conv1_10 = tf.keras.layers.ReLU()(conv1_10)

    conv1_11 = Conv1D(32, 5, padding='same')(conv1_10)
    conv1_11 = BatchNormalization()(conv1_11)

    concat_5 = Concatenate()([max_4, conv1_11])
    concat_5 = tf.keras.layers.ReLU()(concat_5)
    max_5 = MaxPool1D(5, strides=2, padding="same")(concat_5)

    flat = Flatten()(max_5)
    dense_1 = Dense(32, activation='relu')(flat)
    dense_2 = Dense(5, activation="softmax")(dense_1)

    model = Model(inputs=input_, outputs=dense_2)

    model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=[tfa.metrics.F1Score(2, "micro")])
    return model


checkpoint_cb = ModelCheckpoint("senior_multi_diseases_4.h5", save_best_only=True)

model_conv1d_mit = build_conv1d_model_mit(input_shape=(x_train_mit.shape[1], x_train_mit.shape[2]))
model_conv1d_mit.summary()
history_conv1d_mit = model_conv1d_mit.fit(x_train_mit, y_train_mit, epochs=50, batch_size=300,
                                          validation_data=(x_valid_mit, y_valid_mit), callbacks=[checkpoint_cb])
model_conv1d_mit.load_weights("senior_multi_diseases_4.h5")
model_conv1d_mit.evaluate(x_test_mit, y_test_mit)
conv1d_pred_mit_proba = model_conv1d_mit.predict(x_test_mit)
# model_conv1d_mit.evaluate(x_train_mit, y_train_mit)
# conv1d_pred_mit_proba = model_conv1d_mit.predict (x_train_mit)
conv1d_pred_mit = np.argmax(conv1d_pred_mit_proba, axis=1)
conv1d_pred_proba_mit = model_conv1d_mit.predict(x_test_mit)
# conv1d_pred_proba_mit = model_conv1d_mit.predict (x_train_mit)
conv1d_pred_mit = np.argmax(conv1d_pred_proba_mit, axis=1)
print(classification_report(out_test_mit, conv1d_pred_mit, target_names=[MIT_Outcome[i] for i in MIT_Outcome]))

MIT Train classes: 
 0.0    72471
4.0     6431
2.0     5788
1.0     2223
3.0      641
Name: Class, dtype: int64

MIT Test classes: 
 0.0    18118
4.0     1608
2.0     1448
1.0      556
3.0      162
Name: Class, dtype: int64
(87554, 188)
(21892, 188)


/usr/local/lib/python3.10/dist-packages/tensorflow_addons/utils/tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(


Traing dataset size:  (97892, 187, 1)  -- Y size:  (97892, 5)
Validation dataset size:  (17511, 187, 1)  -- Y size:  (17511, 5)
Test dataset size:  (21892, 187, 1)  -- Y size:  (21892, 5)
Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 187, 1)]     0           []                               
                                                                                                  
 conv1d (Conv1D)                (None, 187, 32)      192         ['input_1[0][0]']                
                                                                                                  
 batch_normalization (BatchNorm  (None, 187, 32)     128         ['conv1d[0][0]']                 
 alization)                                                                                       
     